In [1]:
!pip install -U datasets
!pip install optuna
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [2]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, set_seed, AutoModelForMaskedLM, TrainingArguments, Trainer, AutoConfig, AutoModelForSequenceClassification
import torch
from huggingface_hub import login
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os
os.environ["WANDB_DISABLED"] = "true"
import optuna


from dotenv import load_dotenv
import os

load_dotenv()
login(token=os.getenv("HUGGINGFACE_TOKEN"))

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_train = pd.read_csv("/content/drive/My Drive/tcc/data/train.csv")
train, valid = train_test_split(df_train, test_size = 0.5)

In [5]:
test = pd.read_csv("/content/drive/My Drive/tcc/data/test.csv")
unsup = pd.read_csv("/content/drive/My Drive/tcc/data/unsup.csv")

In [6]:
ds = DatasetDict({
    "train": Dataset.from_pandas(train.reset_index(drop = True)),
    "valid": Dataset.from_pandas(valid.reset_index(drop = True)),
    "test": Dataset.from_pandas(test.reset_index(drop = True)),
    "unsup": Dataset.from_pandas(unsup.reset_index(drop = True)),
})

In [7]:
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
def tokenize(batch):
  return tokenizer(batch["text"], truncation = True, max_length = 128, return_special_tokens_mask = True)

ds_mlm = ds.map(tokenize, batched = True)

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Map:   0%|          | 0/40964 [00:00<?, ? examples/s]

In [9]:
ds_mlm = ds_mlm.remove_columns(["label", "text"])

In [10]:
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15)

In [11]:
training_args = TrainingArguments(
    output_dir = "/content/drive/My Drive/tcc/domain_adaptation/training_args",
    per_device_train_batch_size = 32,
    num_train_epochs = 6,
    logging_strategy = "epoch",
    evaluation_strategy = "epoch",
    save_strategy  = "no",
    log_level = "error",
    report_to = "none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer = Trainer(
    model = AutoModelForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased"), processing_class = tokenizer, args = training_args,
    data_collator = data_collator, train_dataset = ds_mlm["unsup"], eval_dataset = ds_mlm["train"]
)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pool

In [13]:
trainer.train()

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,0.455400,0.342925
2,0.348300,0.312602
3,0.319500,0.318551
4,0.295100,0.270581
5,0.279300,0.270364
6,0.270300,0.268640


TrainOutput(global_step=7686, training_loss=0.3279939006772663, metrics={'train_runtime': 6233.4731, 'train_samples_per_second': 39.43, 'train_steps_per_second': 1.233, 'total_flos': 1.6172745508491264e+16, 'train_loss': 0.3279939006772663, 'epoch': 6.0})

In [14]:
trainer.model.save_pretrained("/content/drive/My Drive/tcc/domain_adaptation")
tokenizer.save_pretrained("/content/drive/My Drive/tcc/domain_adaptation")

('/content/drive/My Drive/tcc/domain_adaptation/tokenizer_config.json',
 '/content/drive/My Drive/tcc/domain_adaptation/special_tokens_map.json',
 '/content/drive/My Drive/tcc/domain_adaptation/vocab.txt',
 '/content/drive/My Drive/tcc/domain_adaptation/added_tokens.json',
 '/content/drive/My Drive/tcc/domain_adaptation/tokenizer.json')

In [15]:
def tonekize(batch):
  return tokenizer(batch["text"], truncation = True, max_length = 128)

ds_enc = ds.map(tokenize, batched = True)
ds_enc = ds_enc.remove_columns(["text"])

Map:   0%|          | 0/155 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Map:   0%|          | 0/40964 [00:00<?, ? examples/s]

In [16]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/My Drive/tcc/domain_adaptation")

In [17]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [18]:
config = AutoConfig.from_pretrained("/content/drive/My Drive/tcc/domain_adaptation")
config.num_labels = 2

In [23]:
def objective(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)
    weight_decay = trial.suggest_categorical("weight_decay", [0.0, 0.1, 0.01])
    lr_scheduler_type = trial.suggest_categorical("lr_scheduler_type", ["linear", "constant", "constant_with_warmup"])

    training_args = TrainingArguments(
        output_dir = "/content/drive/My Drive/tcc/fine_tuning",
        num_train_epochs=3,
        learning_rate = learning_rate,
        lr_scheduler_type = lr_scheduler_type,
        per_device_train_batch_size = 32,
        per_device_eval_batch_size = 32,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        weight_decay = weight_decay,
        logging_strategy = "epoch",
        load_best_model_at_end = True,
        metric_for_best_model = "eval_accuracy",
        save_total_limit = 1,
        log_level = "error",
        report_to = "none",
    )
    trainer = Trainer(
        model = model,
        processing_class = tokenizer,
        args = training_args,
        compute_metrics = compute_metrics,
        train_dataset = ds_enc["train"],
        eval_dataset = ds_enc["valid"],
        )

    trainer.train()
    metrics = trainer.evaluate()
    return metrics["eval_accuracy"]

In [24]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)


print("Melhores Hiperparâmetros:")
print(study.best_params)

[I 2025-02-13 13:07:47,119] A new study created in memory with name: no-name-82f147de-5387-4885-aae4-e97f0ee99e11
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.6983, 'grad_norm': 2.238696813583374, 'learning_rate': 1.4031597692449143e-05, 'epoch': 1.0}
{'eval_loss': 0.6925143003463745, 'eval_accuracy': 0.4935897435897436, 'eval_precision': 0.5306807286673059, 'eval_recall': 0.5052631578947369, 'eval_f1': 0.3704214559386973, 'eval_runtime': 0.906, 'eval_samples_per_second': 172.19, 'eval_steps_per_second': 5.519, 'epoch': 1.0}
{'loss': 0.6822, 'grad_norm': 2.4323508739471436, 'learning_rate': 1.4031597692449143e-05, 'epoch': 2.0}
{'eval_loss': 0.6879183053970337, 'eval_accuracy': 0.5256410256410257, 'eval_precision': 0.5385551948051948, 'eval_recall': 0.53125, 'eval_f1': 0.5048043925875085, 'eval_runtime': 0.9005, 'eval_samples_per_second': 173.237, 'eval_steps_per_second': 5.552, 'epoch': 2.0}
{'loss': 0.6716, 'grad_norm': 1.5661444664001465, 'learning_rate': 1.4031597692449143e-05, 'epoch': 3.0}
{'eval_loss': 0.6851727962493896, 'eval_accuracy': 0.5512820512820513, 'eval_precision': 0.5754310344827587, 'eval_recall': 0.55756578947

[I 2025-02-13 13:08:35,716] Trial 0 finished with value: 0.5512820512820513 and parameters: {'learning_rate': 1.4031597692449143e-05, 'num_train_epochs': 6, 'weight_decay': 0.1, 'lr_scheduler_type': 'constant_with_warmup'}. Best is trial 0 with value: 0.5512820512820513.


{'eval_loss': 0.6851727962493896, 'eval_accuracy': 0.5512820512820513, 'eval_precision': 0.5754310344827587, 'eval_recall': 0.5575657894736843, 'eval_f1': 0.5260416666666666, 'eval_runtime': 0.96, 'eval_samples_per_second': 162.494, 'eval_steps_per_second': 5.208, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

{'eval_loss': 2.212043046951294, 'eval_accuracy': 0.6153846153846154, 'eval_precision': 0.6216032608695652, 'eval_recall': 0.6177631578947369, 'eval_f1': 0.6130952380952381, 'eval_runtime': 0.9797, 'eval_samples_per_second': 159.225, 'eval_steps_per_second': 5.103, 'epoch': 3.0}


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'loss': 0.0001, 'grad_norm': 0.006454790011048317, 'learning_rate': 4.3933129291242366e-05, 'epoch': 1.0}
{'eval_loss': 2.9252021312713623, 'eval_accuracy': 0.6153846153846154, 'eval_precision': 0.625, 'eval_recall': 0.6111842105263158, 'eval_f1': 0.6025815217391304, 'eval_runtime': 0.9593, 'eval_samples_per_second': 162.622, 'eval_steps_per_second': 5.212, 'epoch': 1.0}
{'loss': 0.1018, 'grad_norm': 90.6822280883789, 'learning_rate': 4.3933129291242366e-05, 'epoch': 2.0}
{'eval_loss': 3.1087114810943604, 'eval_accuracy': 0.5961538461538461, 'eval_precision': 0.604201680672269, 'eval_recall': 0.5917763157894737, 'eval_f1': 0.5816982336667376, 'eval_runtime': 0.9589, 'eval_samples_per_second': 162.68, 'eval_steps_per_second': 5.214, 'epoch': 2.0}
{'loss': 0.4717, 'grad_norm': 44.151798248291016, 'learning_rate': 4.3933129291242366e-05, 'epoch': 3.0}
{'eval_loss': 2.75797438621521, 'eval_accuracy': 0.5705128205128205, 'eval_precision': 0.5750700280112044, 'eval_recall': 0.56611842105263

[I 2025-02-13 13:13:33,325] Trial 9 finished with value: 0.6153846153846154 and parameters: {'learning_rate': 4.3933129291242366e-05, 'num_train_epochs': 8, 'weight_decay': 0.01, 'lr_scheduler_type': 'constant'}. Best is trial 2 with value: 0.6282051282051282.


{'eval_loss': 2.9252021312713623, 'eval_accuracy': 0.6153846153846154, 'eval_precision': 0.625, 'eval_recall': 0.6111842105263158, 'eval_f1': 0.6025815217391304, 'eval_runtime': 0.9988, 'eval_samples_per_second': 156.192, 'eval_steps_per_second': 5.006, 'epoch': 3.0}
Melhores Hiperparâmetros:
{'learning_rate': 1.2472328789827284e-05, 'num_train_epochs': 8, 'weight_decay': 0.01, 'lr_scheduler_type': 'constant'}


In [27]:
training_args_fine_tune = TrainingArguments(
    output_dir = "/content/drive/My Drive/tcc/fine_tuning",
    num_train_epochs = 6,
    learning_rate = 3.415422640696538e-05,
    lr_scheduler_type = "constant_with_warmup",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    weight_decay= 0.0,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    save_total_limit=1, log_level='error')

In [28]:
trainer = Trainer(
    model=model,
    processing_class =tokenizer,
    args=training_args_fine_tune,
    compute_metrics=compute_metrics,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["valid"],
)

In [29]:
trainer.train()

{'loss': 1.037, 'grad_norm': 83.00989532470703, 'learning_rate': 3.415422640696538e-05, 'epoch': 1.0}
{'eval_loss': 1.0543079376220703, 'eval_accuracy': 0.5897435897435898, 'eval_precision': 0.5912698412698413, 'eval_recall': 0.5907894736842105, 'eval_f1': 0.5894736842105264, 'eval_runtime': 0.9117, 'eval_samples_per_second': 171.101, 'eval_steps_per_second': 5.484, 'epoch': 1.0}
{'loss': 0.4971, 'grad_norm': 57.23299026489258, 'learning_rate': 3.415422640696538e-05, 'epoch': 2.0}
{'eval_loss': 1.2941458225250244, 'eval_accuracy': 0.5897435897435898, 'eval_precision': 0.5951086956521738, 'eval_recall': 0.5921052631578947, 'eval_f1': 0.5873015873015873, 'eval_runtime': 0.9188, 'eval_samples_per_second': 169.783, 'eval_steps_per_second': 5.442, 'epoch': 2.0}
{'loss': 0.4125, 'grad_norm': 76.40199279785156, 'learning_rate': 3.415422640696538e-05, 'epoch': 3.0}
{'eval_loss': 1.5018998384475708, 'eval_accuracy': 0.5833333333333334, 'eval_precision': 0.5845556514236411, 'eval_recall': 0.5805

TrainOutput(global_step=234, training_loss=0.40509392257429594, metrics={'train_runtime': 64.0079, 'train_samples_per_second': 14.529, 'train_steps_per_second': 3.656, 'train_loss': 0.40509392257429594, 'epoch': 6.0})

In [30]:
pred = trainer.predict(ds_enc["test"])
metrics = compute_metrics(pred)
print(f"Metrics on test set: {metrics}")

Metrics on test set: {'accuracy': 0.6282051282051282, 'precision': 0.6027777777777779, 'recall': 0.5761839396019217, 'f1': 0.5685676139614724}


In [31]:
model_no_adapt = AutoModelForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", config=config)

In [32]:
trainer_no_adapt = Trainer(
    model = model_no_adapt,
    processing_class = tokenizer,
    args = training_args_fine_tune,
    compute_metrics = compute_metrics,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["valid"],
)

In [33]:
trainer_no_adapt.train()

{'loss': 0.7271, 'grad_norm': 3.3815550804138184, 'learning_rate': 3.415422640696538e-05, 'epoch': 1.0}
{'eval_loss': 0.6944980621337891, 'eval_accuracy': 0.5, 'eval_precision': 0.5381944444444444, 'eval_recall': 0.5108552631578948, 'eval_f1': 0.3988142292490119, 'eval_runtime': 0.9298, 'eval_samples_per_second': 167.772, 'eval_steps_per_second': 5.377, 'epoch': 1.0}
{'loss': 0.7094, 'grad_norm': 3.4146125316619873, 'learning_rate': 3.415422640696538e-05, 'epoch': 2.0}
{'eval_loss': 0.6927799582481384, 'eval_accuracy': 0.5512820512820513, 'eval_precision': 0.6497584541062802, 'eval_recall': 0.5611842105263157, 'eval_f1': 0.47930574098798395, 'eval_runtime': 0.9322, 'eval_samples_per_second': 167.339, 'eval_steps_per_second': 5.363, 'epoch': 2.0}
{'loss': 0.6903, 'grad_norm': 10.40592098236084, 'learning_rate': 3.415422640696538e-05, 'epoch': 3.0}
{'eval_loss': 0.7114967107772827, 'eval_accuracy': 0.5448717948717948, 'eval_precision': 0.6164021164021164, 'eval_recall': 0.554276315789473

TrainOutput(global_step=234, training_loss=0.6581523520314795, metrics={'train_runtime': 64.7259, 'train_samples_per_second': 14.368, 'train_steps_per_second': 3.615, 'train_loss': 0.6581523520314795, 'epoch': 6.0})

In [34]:
pred_no_adapt = trainer_no_adapt.predict(ds_enc["test"])
metrics_no_adapt = compute_metrics(pred_no_adapt)
print(f"Metrics without Domain Adaptation: {metrics_no_adapt}")

Metrics without Domain Adaptation: {'accuracy': 0.5769230769230769, 'precision': 0.5796052631578947, 'recall': 0.5830473575840769, 'f1': 0.5734879867439934}
